# Library

In [31]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools
import seaborn as sns
import pickle as pkl
from sklearn import tree as tr
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



# Data

In [32]:

df = pd.read_csv("/Users/sylvainestebe/Code/data_child/data/dataset.csv")

data = {
# demographic data
        'Age': df.Age,
        'Region': df.Dep,
        'Postcode': df.CodePostalBaseline,
#        'NumberChildren' : df.NumberChildren,
        'SingleMom' : df.SingleMum,
# knowledge
        'NumberKnown' :df.KnownNbTypeECS,

# output
        'Intention' : df.Intention,
        'Applied' : df.ECSApp,
        'Use' : df.ECSUseYes
}
 
df = pd.DataFrame(data)

/var/folders/td/5r8mr8t14cn_c4w5c8ty48lw0000gn/T/ipykernel_75244/3063411514.py:1: DtypeWarning: Columns (96,111) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/sylvainestebe/Code/data_child/data/dataset.csv")


In [33]:
# Replace "Yes" with 1 and "No" with 0 in the 'col_string' column
df.Intention = df.Intention.replace({'Yes': 1, 'No': 0})
df.SingleMom = df.SingleMom.replace({True:1,False:0})

In [34]:
# Look is there are Nan value
print(df.isna().sum())


Age            0
Region         0
Postcode       0
SingleMom      0
NumberKnown    0
Intention      0
Applied        0
Use            0
dtype: int64


# Model

In [36]:
## Train-Test

# X = df.iloc[:,:9].values
X = df.iloc[:,:5].values

y = (df.iloc[:,6]+ 2 * df.iloc[:,7])
y = y.values 
np.unique(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.15, 
                                                    random_state=42)

# let's further split the training set into a training and a validation set (15%)
X_train, X_val, y_train, y_val = train_test_split(X_train, 
                                                  y_train,
                                                  test_size=X_test.shape[0] / X_train.shape[0],  
                                                  random_state=42)

for x,y,name in zip([X_train, X_val, X_test],
                    [y_train, y_val, y_test],
                    ['train', 'val', 'test']):
    df = pd.DataFrame(np.hstack([x,y.reshape(-1,1)]))
    df.columns = df.columns
    df.to_csv(f'../data/{name}.csv')